In [1]:
!pip install --upgrade --force-reinstall numpy pandas scikit-learn gensim datasets nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 646.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [16]:
from datasets import load_dataset
dataset = load_dataset("trec")  # No need for trust_remote_code
train_data = dataset['train']
test_data = dataset['test']
train_texts = train_data['text']
train_labels = train_data['coarse_label']
test_texts = test_data['text']
test_labels = test_data['coarse_label']
print("Sample Question:", train_data[0]['text'])
print("Label:", train_data[0]['coarse_label'])

Sample Question: How did serfdom develop in and then leave Russia ?
Label: 2


In [17]:

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
import numpy as np

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()
lancaster = LancasterStemmer()

def preprocess(text, method='none'):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [w for w in tokens if w.isalpha() and w not in stop_words]
    if method == 'porter':
        tokens = [porter.stem(w) for w in tokens]
    elif method == 'lancaster':
        tokens = [lancaster.stem(w) for w in tokens]
    elif method == 'lemmatizer':
        tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return tokens


In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from gensim.models import Word2Vec, FastText

def run_vectorizer_model(preproc_method, vectorizer_type='bow', classifier_type='logreg'):
    processed_train = [preprocess(t, preproc_method) for t in train_texts]
    processed_test = [preprocess(t, preproc_method) for t in test_texts]

    if vectorizer_type == 'bow':
        vectorizer = CountVectorizer()
        X_train_vec = vectorizer.fit_transform([" ".join(t) for t in processed_train])
        X_test_vec = vectorizer.transform([" ".join(t) for t in processed_test])

    elif vectorizer_type == 'tfidf':
        vectorizer = TfidfVectorizer()
        X_train_vec = vectorizer.fit_transform([" ".join(t) for t in processed_train])
        X_test_vec = vectorizer.transform([" ".join(t) for t in processed_test])

    elif vectorizer_type == 'word2vec':
        model = Word2Vec(sentences=processed_train, vector_size=100, window=5, min_count=1, workers=4)
        X_train_vec = np.array([np.mean([model.wv[w] for w in words if w in model.wv] or [np.zeros(100)], axis=0)
                                for words in processed_train])
        X_test_vec = np.array([np.mean([model.wv[w] for w in words if w in model.wv] or [np.zeros(100)], axis=0)
                               for words in processed_test])

    elif vectorizer_type == 'fasttext':
        model = FastText(sentences=processed_train, vector_size=100, window=5, min_count=1, workers=4)
        X_train_vec = np.array([np.mean([model.wv[w] for w in words if w in model.wv] or [np.zeros(100)], axis=0)
                                for words in processed_train])
        X_test_vec = np.array([np.mean([model.wv[w] for w in words if w in model.wv] or [np.zeros(100)], axis=0)
                               for words in processed_test])
    else:
        return None

    if classifier_type == 'logreg':
        model = LogisticRegression(max_iter=1000)
    elif classifier_type == 'tree':
        model = DecisionTreeClassifier()
    elif classifier_type == 'forest':
        model = RandomForestClassifier()
    elif classifier_type == 'mlp':
        model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300)
    else:
        return None

    model.fit(X_train_vec, train_labels)
    preds = model.predict(X_test_vec)
    acc = accuracy_score(test_labels, preds)
    return acc


In [19]:
results = []
methods = ['none', 'porter', 'lancaster', 'lemmatizer']
vectorizers = ['bow', 'tfidf', 'word2vec', 'fasttext']
classifiers = ['logreg', 'tree', 'forest', 'mlp']

for method in methods:
    for vec in vectorizers:
        for clf in classifiers:
            try:
                acc = run_vectorizer_model(method, vec, clf)
                results.append((vec, method, clf, acc))
                print(f"{vec} + {method} + {clf} => Accuracy: {acc:.4f}")
            except Exception as e:
                print(f"Failed: {vec} + {method} + {clf} with error: {e}")

df_results = pd.DataFrame(results, columns=['Vectorizer', 'Preprocessing', 'Classifier', 'Accuracy'])
df_results.sort_values(by='Accuracy', ascending=False)

bow + none + logreg => Accuracy: 0.7560
bow + none + tree => Accuracy: 0.7280
bow + none + forest => Accuracy: 0.7260
bow + none + mlp => Accuracy: 0.7280
tfidf + none + logreg => Accuracy: 0.7600
tfidf + none + tree => Accuracy: 0.7400
tfidf + none + forest => Accuracy: 0.7320
tfidf + none + mlp => Accuracy: 0.7240
word2vec + none + logreg => Accuracy: 0.2040
word2vec + none + tree => Accuracy: 0.4060
word2vec + none + forest => Accuracy: 0.5320


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


word2vec + none + mlp => Accuracy: 0.5080
fasttext + none + logreg => Accuracy: 0.3500
fasttext + none + tree => Accuracy: 0.3240
fasttext + none + forest => Accuracy: 0.4720


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


fasttext + none + mlp => Accuracy: 0.4220
bow + porter + logreg => Accuracy: 0.7520
bow + porter + tree => Accuracy: 0.7180
bow + porter + forest => Accuracy: 0.7200
bow + porter + mlp => Accuracy: 0.6940
tfidf + porter + logreg => Accuracy: 0.7500
tfidf + porter + tree => Accuracy: 0.7080
tfidf + porter + forest => Accuracy: 0.7320
tfidf + porter + mlp => Accuracy: 0.6680
word2vec + porter + logreg => Accuracy: 0.4160
word2vec + porter + tree => Accuracy: 0.4200
word2vec + porter + forest => Accuracy: 0.5620


...

fasttext + lancaster + mlp => Accuracy: 0.3600
bow + lemmatizer + logreg => Accuracy: 0.7540
bow + lemmatizer + tree => Accuracy: 0.7220
bow + lemmatizer + forest => Accuracy: 0.7120
bow + lemmatizer + mlp => Accuracy: 0.7140
tfidf + lemmatizer + logreg => Accuracy: 0.7480
tfidf + lemmatizer + tree => Accuracy: 0.7260
tfidf + lemmatizer + forest => Accuracy: 0.7360
tfidf + lemmatizer + mlp => Accuracy: 0.6800
word2vec + lemmatizer + logreg => Accuracy: 0.2140
word2vec + lemmatizer + tree => Accuracy: 0.4080
word2vec + lemmatizer + forest => Accuracy: 0.5200


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


word2vec + lemmatizer + mlp => Accuracy: 0.5440
fasttext + lemmatizer + logreg => Accuracy: 0.3380
fasttext + lemmatizer + tree => Accuracy: 0.3180
fasttext + lemmatizer + forest => Accuracy: 0.4500
fasttext + lemmatizer + mlp => Accuracy: 0.4060


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


,Vectorizer,Preprocessing,Classifier,Accuracy
4,tfidf,none,logreg,0.760
0,bow,none,logreg,0.756
48,bow,lemmatizer,logreg,0.754
16,bow,porter,logreg,0.752
20,tfidf,porter,logreg,0.750
...,...,...,...,...
13,fasttext,none,tree,0.324
45,fasttext,lancaster,tree,0.320
61,fasttext,lemmatizer,tree,0.318
56,word2vec,lemmatizer,logreg,0.214
